In [ ]:
# default_exp core.scraping.event

# Event

> Scrapes CZ event information.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

from czapi.core.scraping.base import make_soup
from bs4 import BeautifulSoup
from typing import Union, Optional

In [ ]:
#hide

url = 'https://www.curlingzone.com/event.php?view=Main&eventid=6938'
soup = make_soup(url)

In [ ]:
# export

def _get_event_name_from_linescore_page(
     soup : BeautifulSoup
)->str:
    """Returns the name of the event based on the event_id. Built so for larger-scale scraping less API calls are made."""
    return soup.find('title').string

def _get_event_name_from_boxscore_page(
    
    soup : BeautifulSoup

)->str:
    """Returns the name of the vent based on the game_id. Build so for larger-scale scrpaing less API calls are made."""
    
    return soup.find('h3',attrs={'class':'entry-title-widget'}).string

In [ ]:
# export


def _get_event_url(
     cz_event_id : Union[str,int]
)->str:
    """Returns the cz event page url."""
    
    return 'https://curlingzone.com/event.php?view=Main&eventid=%s#1'%cz_event_id

def _get_game_url(
    cz_game_id : Union[str,int]
)->str:
    """Returns the cz game page url."""

    return 'https://www.curlingzone.com/game.php?1=1&showgameid=%s#1'%cz_game_id

def get_event_name(
     cz_event_id : Optional[Union[str,int]] = None
    ,cz_game_id : Optional[Union[str,int]] = None
    ,**request_kwargs
)->str:
    """Returns the name of the event based on the cz_event_id or the cz_game_id."""
    
    # TODO : how to handle situations where an event_id is valid but that event hasn't been conducted yet?
    # e.g. event_id = 12312321312 returns 'CurlingZone – Everything Curling'
    
    ids = [cz_event_id,cz_game_id]
    if all(ids) or not any(ids):
        raise ValueError("One of cz_event_id or cz_game_id can be a non NoneType.")
    
    if cz_event_id:
        url = _get_event_url(cz_event_id = cz_event_id)
        f = _get_event_name_from_linescore_page
    else:
        url = _get_game_url(cz_game_id = cz_game_id)
        f = _get_event_name_from_boxscore_page
        
    soup = make_soup(url=url,**request_kwargs)
    
    return f(soup=soup)

In [ ]:
#hide
soup = make_soup('https://curlingzone.com/event.php?view=Main&eventid=6900#1')
soup.find('title').string

'Scotties Tournament of Hearts'

In [ ]:
# tests

# scotties 
assert get_event_name(cz_event_id = '6900') == 'Scotties Tournament of Hearts'
assert get_event_name(cz_game_id = 297288) == 'Scotties Tournament of Hearts'

# random event
assert get_event_name(cz_event_id = '6938') == 'Hokkaido Bank Curling Classic'
assert get_event_name(cz_game_id = 284807) == 'Hokkaido Bank Curling Classic'

try:
    get_event_name(1,2)
except ValueError as e:
    assert isinstance(e,ValueError)
    
try:
    get_event_name()
except ValueError as e:
    assert isinstance(e,ValueError)

In [ ]:
# export

def _get_event_date_from_linescore_page(

     soup : BeautifulSoup
)->str:
    """Returns the dates of the event from the passed soup. Built so for larger-scale scraping less API calls are made."""
    return soup.find(name='div',attrs={'class':'badge-widget'}).string

def _get_event_date_from_boxscore_page(

    soup : BeautifulSoup

)->str:
    """Returns the dates of the event from the passed soup. Built so for larger-scale scraping less API calls are made."""
    return soup.find('div',attrs={'class':'badge-widget'}).string

In [ ]:
# export

def get_event_date(

     cz_event_id : Optional[Union[str,int]] = None
    ,cz_game_id : Optional[Union[str,int]] = None
    ,**request_kwargs

)->str:
    """Returns the dates of the event based on the cz_event_id or the cz_game_id."""
    
    # TODO : how to handle situations where an event_id is valid but that event hasn't been conducted yet?
    # e.g. event_id = 12312321312 returns 'Recent'
    
    ids = [cz_event_id,cz_game_id]
    if all(ids) or not any(ids):
        raise ValueError("One of cz_event_id or cz_game_id can be a non NoneType.")
    
    if cz_event_id:
        url = _get_event_url(cz_event_id = cz_event_id)
        f = _get_event_date_from_linescore_page
    else:
        url = _get_game_url(cz_game_id = cz_game_id)
        f = _get_event_date_from_boxscore_page
        
    soup = make_soup(url=url,**request_kwargs)
    
    return f(soup=soup)

In [ ]:
# tests

# scotties
assert get_event_date(cz_event_id = '6900') == 'Jan 28 - Feb 6, 2022'
assert get_event_date(cz_game_id = 297288) == 'Jan 28 - Feb 6, 2022'

try:
    get_event_date()
except ValueError as e:
    assert isinstance(e,ValueError)

try:
    get_event_date(1,2)
except ValueError as e:
    assert isinstance(e,ValueError)
    
# random event
assert get_event_date(cz_event_id = '6938') == 'Aug 19 - 22, 2021'
assert get_event_date(cz_game_id = 284807) == 'Aug 19 - 22, 2021'